In [3]:
import torch
torch.__version__

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
x = torch.rand(5, 3).to(device)

In [6]:
x

# K-Means

In [7]:
import matplotlib.pyplot as plt

In [8]:
N = 100
x1 = torch.randn(N, 2)
x2 = torch.randn(N, 2) + torch.Tensor([3, 3])
x3 = torch.randn(N, 2) + torch.Tensor([-3, 3])

In [9]:
X = torch.cat([x1, x2, x3], dim=0)
print(X.shape)

In [ ]:
for i in range(1):
    plt.scatter(X[i*N:(i+1)*N, 0], X[i*N:(i+1)*N, 1])
plt.show()

In [ ]:
K = 3
centroids = X[torch.randperm(len(X))[:K]]
while True:
    D = torch.cdist(X, centroids)
    clusters = torch.argmin(D, dim=1)
    for i in range(3):
        plt.scatter(X[i*N:(i+1)*N, 0], X[i*N:(i+1)*N, 1])
    old_centroids = centroids.clone()
    for k in range(K):
        plt.scatter(centroids[k, 0], centroids[k, 1], marker='x')
        centroids[k] = X[clusters == k].mean(dim=0)
    change = torch.sum((centroids - old_centroids) ** 2)
    if change < 0.1:
        break
    plt.show()

In [ ]:
import time
device = 'cuda'

N = 100000
x1 = torch.randn(N, 2)
x2 = torch.randn(N, 2) + torch.Tensor([3, 3])
x3 = torch.randn(N, 2) + torch.Tensor([-3, 3])
X = torch.cat([x1, x2, x3], dim=0).to(device)

t1 = time.time()
K = 3
centroids = X[torch.randperm(len(X))[:K]]
for _ in range(100):
    D = torch.cdist(X, centroids)
    clusters = torch.argmin(D, dim=1)
    for k in range(K):
        centroids[k] = X[clusters == k].mean(dim=0)
print(time.time() - t1)

# Auto differentiation

In [ ]:
x = torch.tensor(3., requires_grad=True)
y = x**2 + 2*x + 1 
# y' = 2x + 2
y.backward()

In [ ]:
x.grad # 2(3) + 2

# Regression

In [ ]:
N = 100
x = torch.rand(N, 1)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1)
plt.scatter(x, y)

In [ ]:
ones = torch.ones(N, 1)
X = torch.cat([x, ones], dim=1)

In [ ]:
R = torch.inverse(X.T @ X) @ X.T @ y

In [ ]:
x_ = torch.linspace(0, 1, 100).reshape(-1, 1)

In [ ]:
y_ = x_ * R[0] + R[1]

In [ ]:
plt.scatter(x, y)
plt.scatter(x_, y_, color='r')
plt.show()

nonlinear

In [ ]:
ones = torch.ones(N, 1)
X = torch.cat([x**2, x, ones], dim=1)

In [ ]:
R = torch.inverse(X.T @ X) @ X.T @ y

In [ ]:
y_ = x_**2 * R[0] + x_ * R[1] + R[2]

In [ ]:
plt.scatter(x, y)
plt.scatter(x_, y_, color='r')
plt.show()

In [ ]:
device = 'cpu'

N = 100000000
x = torch.rand(N, 1).to(device)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1).to(device)

t1 = time.time()
degree = 1
ones = torch.ones(N, 1).to(device)
X = torch.cat([x, ones], dim=1)
for i in range(2, degree + 1):
    X = torch.cat([x**i, X], dim=1)
R = torch.inverse(X.T @ X) @ X.T @ y
print(time.time() - t1)

# MLP

In [ ]:
from torch import nn

In [ ]:
# [1] 32 16 [1]
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return self.fc3(x)

In [ ]:
(1*32+32) + (32*16+16) + (16*1+1)

In [ ]:
model = Regressor()

In [ ]:
gen = iter(model.parameters())

In [ ]:
next(gen).shape

In [ ]:
N = 1000
x = torch.rand(N, 1).to(device)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1).to(device)

In [ ]:
device = 'cuda' 

model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(5000):
    y_ = model(x.to(device))
    loss = criterion(y_, y.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [ ]:
x_ = torch.linspace(0, 1, 100).reshape(-1, 1)
model.eval()
with torch.no_grad():
    y_ = model(x_.to(device))
plt.scatter(x, y)
plt.scatter(x_, y_.cpu(), color='r')
plt.show()

# Classification

In [ ]:
N

In [ ]:
X.shape

In [ ]:
Y = torch.cat([torch.zeros(N), torch.ones(N), torch.ones(N) * 2]).long()

In [ ]:
Y

In [ ]:
X.shape[1]

In [ ]:
len(Y.unique())

In [1]:
# [2] 32 16 [3]
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

In [ ]:
model = Classifier()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(5000):
    y_ = model(X.to(device))
    loss = criterion(y_, Y.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [ ]:
model.eval()
with torch.no_grad():
    y_ = model(X.to(device))
acc = torch.sum(Y == y_.argmax(dim=1).cpu()) / len(Y)
print(acc.item())

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
df = load_iris(as_frame=True)

In [ ]:
itrain = np.r_[0:25, 50:75, 100:125]
itest = np.r_[25:50, 75:100, 125:150]
xtrain = df['data'].iloc[itrain].values
xtest = df['data'].iloc[itest].values
ytrain = df['target'].iloc[itrain].values
ytest = df['target'].iloc[itest].values

In [ ]:
xtrain.shape

In [ ]:
len(np.unique(ytrain))

In [ ]:
# [4] 128 [3]
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 3)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.softmax(self.fc2(x), dim=1)

In [ ]:
xtrain_ = torch.tensor(xtrain, dtype=torch.float32)

In [ ]:
xtest_ = torch.tensor(xtest, dtype=torch.float32)

In [ ]:
ytrain_ = torch.tensor(ytrain, dtype=torch.long)

In [ ]:
ytest_ = torch.tensor(ytest, dtype=torch.long)

In [ ]:
model = Classifier()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(5000):
    y_ = model(xtrain_.to(device))
    loss = criterion(y_, ytrain_.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [ ]:
model.eval()
with torch.no_grad():
    y_ = model(xtest_.to(device))
acc = torch.sum(ytest_ == y_.argmax(dim=1).cpu()) / len(y_)
print(acc.item())

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model.state_dict()

In [ ]:
df['data'].columns

In [ ]:
df['data'].max()

In [ ]:
import seaborn as sns

In [ ]:
df = sns.load_dataset('titanic')
df

In [ ]:
df.nunique()

In [ ]:
df = df.drop(columns=['class', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'])

In [ ]:
import pandas as pd

In [ ]:
df = pd.get_dummies(df, dtype=float)

In [ ]:
df

In [ ]:
df = df.drop(columns=['sex_female'])

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
X = df.iloc[:, 1:]
Y = df.iloc[:, 0]

In [ ]:
X

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)

In [ ]:
xtrain = torch.tensor(xtrain.values, dtype=torch.float32)
xtest = torch.tensor(xtest.values, dtype=torch.float32)
ytrain = torch.tensor(ytrain.values, dtype=torch.float32)
ytest = torch.tensor(ytest.values, dtype=torch.float32)
ytrain = ytrain.reshape(-1, 1)
ytest = ytest.reshape(-1, 1)

In [ ]:
xtrain.shape

In [ ]:
ytrain.unique()

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(12, 128)
        self.fc2 = nn.Linear(128, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

In [ ]:
model = Classifier()

In [ ]:
ytrain.shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(5000):
    y_ = model(xtrain.to(device))
    loss = criterion(y_, ytrain.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [ ]:
model.eval()
with torch.no_grad():
    y_ = model(xtest.to(device))
acc = torch.sum(ytest == y_.round().cpu()) / len(y_)
print(acc.item())